## Install before starting

In [1]:
pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install --upgrade openai


     ---------------------------------------- 0.0/65.2 kB ? eta -:--:--
     ------------------------- -------------- 41.0/65.2 kB 2.0 MB/s eta 0:00:01
     ---------------------------------------- 65.2/65.2 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------------------- 57.7/57.7 kB 3.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/644.8 kB ? eta -:--:--
   ------ --------------------------------- 112.6/644.8 kB 6.4 MB/s eta 0:00:01
   ----------------- ---------------------- 286.7/644.8 kB 3.5 MB/s eta 0:00:01
   --------------------------- ------------ 450.6/644.8 kB 4.0 MB/s eta 0:00:01
   ---------------------------------------- 644.8/644.8 kB 4.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/208.0 kB ? eta -:--:--
   ---------------------------------------- 208.0/208.0 kB ? eta 0:00:00
   ---------------------------------------- 0.0/443.6 kB ? eta -:--:--
   -------


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip show openai


Name: openai
Version: 1.74.0
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: Apache-2.0
Location: c:\Users\eynavoz\AppData\Local\Programs\Python\Python312\Lib\site-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## Import

In [51]:
from openai import AzureOpenAI
import os
import pandas as pd
import re
import time
#import gt
from dotenv import load_dotenv
load_dotenv()


True

## open AI connection

In [18]:
AZURE_OPENAI_API_KEY = os.getenv('CLASS_AZURE_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('SUBSCRIPTION_OPENAI_ENDPOINT')

MODEL_4o = 'gpt-4o-mini'
OPENAI_API_VERSION_4o = '2024-08-01-preview'

MODEL_35 = 'gpt-35-16k'
OPENAI_API_VERSION_35 = '2023-12-01-preview'


TEMP_1 = 0
TEMP_2 = .5
TEMP_3 = .9
MAX_TOKENS = 7000
LOG_PROBS = False




In [ ]:
#Example usage with model 35:
client = AzureOpenAI(
    api_key= AZURE_OPENAI_API_KEY,
    api_version= OPENAI_API_VERSION_35,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
)

sys_prompt = {"role": "system", "content": "You are a dietitian and a nutrition expert."}
user_prompt = {"role": "user", "content": """A restaurant wants to provide nutritional information \
for each dish (menu item) it offers. What information should it provide?
Limit your response to no more than 10 requirements."""}
msg = [sys_prompt, user_prompt]
responses = client.chat.completions.create(messages=msg, model=MODEL_35, temperature=TEMP_1, max_tokens=MAX_TOKENS, logprobs= LOG_PROBS, n=1)
print(responses.choices[0].message.content)

1. Calories: The total number of calories in each dish is essential for customers to make informed choices about their calorie intake.

2. Macronutrients: Include the amounts of carbohydrates, proteins, and fats in each dish. This information is crucial for individuals following specific dietary plans or those looking to balance their macronutrient intake.

3. Fiber: The amount of dietary fiber in each dish is important for promoting digestive health and maintaining satiety.

4. Sugar: Specify the amount of added sugars in each dish, as excessive sugar consumption can contribute to various health issues.

5. Sodium: Provide the sodium content to help individuals monitor their sodium intake, especially for those with high blood pressure or other health conditions.

6. Allergens: Clearly indicate if any common allergens, such as gluten, dairy, nuts, or shellfish, are present in the dish to ensure the safety of customers with food allergies.

7. Serving Size: Clearly state the recommended

In [19]:
# Created function for convenience

def generate_response(version, system_prompt, prompt, model, temp):
    """
    Get a response from the specified OpenAI model using the given prompt.
    
    Args:
        version (str): OPEN_API_VERSION
        system_prompt (str): The system prompt to provide the model
        prompt (str): The prompt to provide to the open AI model
        model (str): The name of ID of the openAI engine to use
        temp (str): the temperature parameter
    """

    client = AzureOpenAI(
        api_key= AZURE_OPENAI_API_KEY,
        api_version= version,
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
    )
    sys_prompt = {"role": "system", "content": system_prompt}
    user_prompt = {"role": "user", "content": prompt}
    msg = [sys_prompt, user_prompt]
    responses = client.chat.completions.create(messages=msg, model=model, temperature=temp, max_tokens=MAX_TOKENS, logprobs= LOG_PROBS, n=1)
    return responses.choices[0].message.content

In [ ]:
#Example usage with model 35:

sys_prompt1 = "You are a dietitian and a nutrition expert."
user_prompt1 = """A restaurant wants to provide nutritional information \
for each dish (menu item) it offers. What information should it provide?
Limit your response to no more than 10 requirements."""
generate_response(OPENAI_API_VERSION_35, sys_prompt1, user_prompt1, MODEL_35, TEMP_1)

'1. Calories: The total number of calories in each dish is essential for customers to make informed choices about their calorie intake.\n\n2. Macronutrients: Include the amounts of carbohydrates, proteins, and fats in each dish. This information is crucial for individuals following specific dietary plans or those looking to balance their macronutrient intake.\n\n3. Fiber: The amount of dietary fiber in each dish is important for customers who want to increase their fiber intake for digestive health and satiety.\n\n4. Sodium: Many people need to monitor their sodium intake due to health conditions like high blood pressure. Including the sodium content in each dish helps customers make lower-sodium choices.\n\n5. Sugar: The amount of added sugars in each dish is important for individuals who are watching their sugar intake, especially those with diabetes or those trying to reduce their overall sugar consumption.\n\n6. Allergens: Clearly indicate if any common allergens, such as gluten, d

## Part 1

In [ ]:
# Read the file, tkts_1.txt

# For each ticket in the file, your program needs to read the ticket and prompt the LLM 
# to determine if the ticket is a single issue or is really 2 different issues.

sys_prompt_part1 = "You are a helpful assistant that classifies support tickets."

user_prompt_part1 = """
You are given a ticket of a software bug that a customer experiences. 
Your task is to determine if a ticket describes a single issue or is really 2 different issues.

Use the following categories for issues classification:
- Interface
- Lacking Feature
- Logic Defect
- Data
- Security and Access Control
- Configuration
- Stability
- performance

Eamples
Here is the tickect to classify:
{ticket}
End of ticket

Be concise and structured in your output. Return your result ONLY in this format:
{{
Issue Count: [1 or 2]
Issue 1: category name
Detail 1: short summary to support your decision
Issue 2: categor name (Only include this if there are 2 issues.)
Detail 2: short summary to support your decision
}}
Do not add any additional text besides this format.

"""

In [ ]:
# Example usage:
ticket = "The app crashes when uploading large files."
print(generate_response(OPENAI_API_VERSION_35, sys_prompt_part1, user_prompt_part1.format(ticket=ticket), MODEL_35, TEMP_1))


{
Issue Count: 1
Issue 1: Stability
Detail 1: The app crashes when uploading large files, indicating a stability issue.
}


In [37]:
def parse_ticket_file(file_path):
    tickets = []

    # Read the file
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # Parse each line
    for line in lines:
        line = line.strip()
        if line:
            match = re.match(r"(\d+):\s*(.+)", line)
            if match:
                number = int(match.group(1))
                ticket = match.group(2)
                tickets.append({'number': number, 'ticket': ticket})
    
    # Create DataFrame
    df = pd.DataFrame(tickets)
    return df

In [41]:
file_path = "./tkts_1.txt"
df = parse_ticket_file(file_path)

In [48]:
df

,number,ticket,llm,class
0,1,The buttons on the dashboard are misaligned af...,{\nIssue Count: 1\nIssue 1: Interface\nDetail ...,no-split
1,2,User roles are not saving correctly; some user...,{\nIssue Count: 2\nIssue 1: Data\nDetail 1: Us...,split
2,3,Clicking ‘Submit’ twice causes duplicate recor...,{\nIssue Count: 1\nIssue 1: Logic Defect\nDeta...,no-split
3,4,"Some reports show incorrect calculations, and ...",{\nIssue Count: 2\nIssue 1: Logic Defect\nDeta...,split


In [43]:
df['llm'] = df['ticket'].apply(lambda x: generate_response(OPENAI_API_VERSION_35, sys_prompt_part1, user_prompt_part1.format(ticket=x), MODEL_35, TEMP_1))

In [46]:
def classify_llm_response(response_text):
    if not isinstance(response_text, str):
        return 'unknown'
    
    match = re.search(r"Issue\s*Count\s*:\s*(\d+)", response_text, re.IGNORECASE)
    
    if match:
        count = int(match.group(1))
        if count == 1:
            return 'no-split'
        elif count == 2:
            return 'split'
    
    return 'unknown'

In [47]:
df['class'] = df['llm'].apply(classify_llm_response)

In [ ]:
tests = [
    {"model": MODEL_35, "version": OPENAI_API_VERSION_35, "temperature": 0},
    {"model": MODEL_4o, "version": OPENAI_API_VERSION_4o,"temperature": 0},
    {"model": MODEL_4o, "version": OPENAI_API_VERSION_4o,"temperature": 0.9}
]

# ==== Output Collector ====
output_lines = []

# ==== Processing ====
for idx, row in df.iterrows():
    ticket_number = row['number']
    ticket_text = row['ticket']
    output_lines.append(f"{ticket_number}:")

    for test in tests:
        model = test['model']
        temp = test['temperature']
        version = test['version']
        try:
            response = generate_response(
                version,  # Or adapt if version changes
                sys_prompt_part1,
                user_prompt_part1.format(ticket=ticket_text),
                model,
                temp
            )
            llm_output = response.strip()
        except Exception as e:
            llm_output = f"ERROR: {e}"

        classification = classify_llm_response(llm_output)
        output_lines.append(f"{model}, {temp}: {classification}")
        output_lines.append(f"LLM response: {llm_output}")

        time.sleep(5)  # Wait to avoid hitting token rate limits
    
    output_lines.append("")  # Empty line between tickets

# ==== Save to File ====
with open("split.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(output_lines))

print("✅ Results written to split.txt")

✅ Results written to split.txt
